In [1]:
from lib import *

# Challenge 25: Break "random access read/write" AES CTR
[Link](https://cryptopals.com/sets/4/challenges/25)

> Back to CTR. Encrypt the recovered plaintext from [this file](https://cryptopals.com/static/challenge-data/25.txt) (the ECB exercise) under CTR with a random key (for this exercise the key should be unknown to you, but hold on to it).  
Now, write the code that allows you to "seek" into the ciphertext, decrypt, and re-encrypt with different plaintext. Expose this as a function, like, "edit(ciphertext, key, offset, newtext)".  
Imagine the "edit" function was exposed to attackers by means of an API call that didn't reveal the key or the original plaintext; the attacker has the ciphertext and controls the offset and "new text".  
Recover the original plaintext.

In [2]:
# Imports
import os
import base64
import itertools

In [3]:
# Given
data = open("25.txt", "r").read()

In [4]:
def edit(ciphertext: bytes, key: bytes, offset: int, newtext: bytes, nonce: int) -> bytes:
    """
    Seek into the ciphertext at the given offset and edit the ciphertext to add the newtext's cipher at the offset.
    """
    keystream = b""
    # Obtain the keystream used to encrypt in the AES CTR Mode.
    # Encrypting newtext to be inserted at offset requires CTR keystream at that offset too.
    stream = CTR_keystream_generator(key, nonce)
    for i in itertools.islice(stream, offset, offset+len(newtext)):
        keystream += i.to_bytes(1, "big")
    
    # Get the cipher for newtext.
    append_cipher = xor_bytes(newtext, keystream)
    
    # Append the cipher of newtext to original cipher.
    result = ciphertext[:offset] + append_cipher
    if len(result) < len(ciphertext):
        return result + ciphertext[len(result):]
    return result

Test the edit function.

In [5]:
random_key = os.urandom(16)
nonce = 0

plaintext = b"hello there"
cipher = CTR(plaintext, random_key, nonce)
print("Original text:", CTR(cipher, random_key, nonce).decode("utf-8"))
edited_cipher = edit(cipher, random_key, 4, b"####", nonce)
print("Edited text:", CTR(edited_cipher, random_key, nonce).decode("utf-8"))

Original text: hello there
Edited text: hell####ere


In [6]:
# If you give text as \x00 it gives out keystream, xors keystream with 0 and thus can decode keystream 
# by using offset as 0.
recovered_bytes = base64.b64decode(data)

random_key = os.urandom(16)
nonce = 0

ciphertext = CTR(recovered_bytes, random_key, nonce)
recovered_keystream = edit(ciphertext, random_key, 0, b'\x00'*len(ciphertext), nonce)
deciphered_bytes = xor_bytes(ciphertext, recovered_keystream)

In [7]:
test(deciphered_bytes == recovered_bytes)

# Challenge 26: CTR bitflipping
[Link](https://cryptopals.com/sets/4/challenges/26)

> There are people in the world that believe that CTR resists bit flipping attacks of the kind to which CBC mode is susceptible.  
Re-implement [the CBC bitflipping exercise from earlier](https://cryptopals.com/sets/2/challenges/16) to use CTR mode instead of CBC mode. Inject an "admin=true" token.

In [8]:
# Imports
import os

In [9]:
# Given
prepend_string = "comment1=cooking%20MCs;userdata="
append_string = ";comment2=%20like%20a%20pound%20of%20bacon"

Function to prepend the URL encoded string to text and encrypt it with CTR.

In [10]:
def encryptor(text: bytes, key: bytes, nonce: int) -> bytes:
    """
    Prepends the string to given text and encrypts with CTR.
    """
    plaintext =  (prepend_string.encode() + text + append_string.encode()).replace(b';', b'";"').replace(b'=', b'"="')
    ciphertext = CTR(plaintext, key, nonce)
    return ciphertext

Function to decrypt the cipihertext and check if "admin=true" is present.

In [11]:
def decryptor(byte_string: bytes, random_key: bytes, nonce: int) -> bool:
    """
    Decrypts the ciphertext via AES CTR Mode and checks if admin is set to true.
    """
    decrypted_string = CTR(byte_string, random_key, nonce)
    if b';admin=true;' in decrypted_string:
        return True
    else:
        return False

In [12]:
target_bytes = b";admin=true;"
random_key = os.urandom(16)
nonce = 0

modified_string = b""

# we take out prefix length and then combine the recovered
# keystream from that offset onwards with inut text to produce
# the required string
prefix_length = len(os.path.commonprefix([encryptor(b'AAAA', random_key, nonce), encryptor(b'', random_key, nonce)]))
print("Prefix length: ", prefix_length)

dummy_input = b"heytheremama"
ciphertext = encryptor(dummy_input, random_key, nonce)
null_cipher = encryptor(b'\x00'*len(ciphertext), random_key, nonce)
recovered_keystream = null_cipher[prefix_length:len(ciphertext)]

injected_bytes = b""
for i in range(len(target_bytes)):
    injected_bytes += (target_bytes[i] ^ recovered_keystream[i]).to_bytes(1, "big")

modified_ciphertext = ciphertext[:prefix_length] + injected_bytes + ciphertext[prefix_length + len(injected_bytes):]

Prefix length:  38


In [13]:
test(decryptor(modified_ciphertext, random_key, nonce) == True)

# Challenge 27: Recover the key from CBC with IV=Key
[Link](https://cryptopals.com/sets/4/challenges/27)

> Take your code from the CBC exercise and modify it so that it repurposes the key for CBC encryption as the IV.  
Applications sometimes use the key as an IV on the auspices that both the sender and the receiver have to know the key already, and can save some space by using it as both a key and an IV.  
Using the key as an IV is insecure; an attacker that can modify ciphertext in flight can get the receiver to decrypt a value that will reveal the key.  
The CBC code from exercise 16 encrypts a URL string. Verify each byte of the plaintext for ASCII compliance (ie, look for high-ASCII values). Noncompliant messages should raise an exception or return an error that includes the decrypted plaintext (this happens all the time in real systems, for what it's worth).  
Use your code to encrypt a message that is at least 3 blocks long:  
**AES-CBC(P_1, P_2, P_3) -> C_1, C_2, C_3**  
Modify the message (you are now the attacker):  
**C_1, C_2, C_3 -> C_1, 0, C_1**  
Decrypt the message (you are now the receiver) and raise the appropriate error if high-ASCII is found.  
As the attacker, recovering the plaintext from the error, extract the key:  
**P'_1 XOR P'_3**

In [14]:
# Imports
import os

In [15]:
# Given
prepend_string = "comment1=cooking%20MCs;userdata="
append_string = ";comment2=%20like%20a%20pound%20of%20bacon"

In [16]:
def check_ascii_compliance(plaintext: bytes) -> bool:
    """
    Returns true if all the characters of plaintext are ASCII compliant (ie are in the ASCII table).
    """
    return all(c < 128 for c in plaintext)

In [17]:
def encryptor(text: bytes, IV: bytes, key: bytes) -> bytes:
    """
    Encrypts the text with AES CBC Mode.
    """
    plaintext = text.replace(b';', b'";"').replace(b'=', b'"="')
    ciphertext = AES_CBC_encrypt(PKCS7_pad(plaintext, len(key)), IV, key)
    return ciphertext

In [18]:
def decryptor(byte_string: bytes, IV: bytes, key: bytes) -> bool:
    """
    Decrypts the ciphertext via AES CBC Mode and checks if all characters are ASCII.
    """
    decrypted_string = AES_CBC_decrypt(byte_string, IV, key)
    print(len(decrypted_string), decrypted_string)
    if not check_ascii_compliance(decrypted_string):
        raise Exception(decrypted_string)

In [19]:
keysize = 16
random_key = os.urandom(keysize)
IV = random_key

plaintext = b"lorem=ipsum;test=fun;padding=dull"
ciphertext = encryptor(plaintext, IV, random_key)
c1 = ciphertext[:keysize]
c2 = ciphertext[keysize:2*keysize]
c3 = ciphertext[2*keysize:]

try:
    decryptor(c1 + b'\x00'*16 + c1, IV, random_key)
except Exception as e:
    decrypted_string = str(e).encode()
    p1 = decrypted_string[:keysize]
    p3 = decrypted_string[2*keysize:]
    decrypted_key = xor_bytes(p1, p3)
    print("> Key found to be:", decrypted_key)

66 b'lorem"="ipsum";"#\xc3\x81\xc2\xa9\\\x02\xc3\xae\xc3\x8b\xc3\xa47\xc2\x9fsnJ\xc2\xb0MV\xc2\xba\xc3\x99)\xc3\x94\xc3\x83\xc2\x9c\x7f\xc3\x80\xc3\x9aX\xc2\x94\xc3\x94l\xc3\xbc\x01\xc3\xa2'
> Key found to be: b'\x1aFU3.9\x15\x12\x0f~\x11\x13@)\x15C'


# Challenge 28: Implement a SHA-1 keyed MAC
[Link](https://cryptopals.com/sets/4/challenges/28)

> Find a SHA-1 implementation in the language you code in.  
Write a function to authenticate a message under a secret key by using a secret-prefix MAC, which is simply:  
**SHA1(key || message)**  
Verify that you cannot tamper with the message without breaking the MAC you've produced, and that you can't produce a new MAC without knowing the secret key.

In [20]:
# Imports
import os
import struct
import hashlib

In [21]:
def left_rotate(value: int, shift: int) -> int:
    """
    Returns value left-rotated by shift bits. In other words, performs a circular shift to the left.
    """
    return ((value << shift) & 0xffffffff) | (value >> (32 - shift))


def sha1(message: bytes, ml=None, h0=0x67452301, h1=0xEFCDAB89, h2=0x98BADCFE, h3=0x10325476, h4=0xC3D2E1F0) -> bytes:
    """
    Returns a string containing the SHA1 hash of the input message. This is a pure python 3 SHA1
    implementation, written starting from the SHA1 pseudo-code on Wikipedia.
    The parameters ml, h0, ..., h5 are for the next challenge.
    """
    
    # Pre-processing:
    if ml is None:
        ml = len(message) * 8

    message += b'\x80'
    while (len(message) * 8) % 512 != 448:
        message += b'\x00'

    message += struct.pack('>Q', ml)

    # Process the message in successive 512-bit chunks:
    for i in range(0, len(message), 64):

        # Break chunk into sixteen 32-bit big-endian integers w[i]
        w = [0] * 80
        for j in range(16):
            w[j] = struct.unpack('>I', message[i + j * 4:i + j * 4 + 4])[0]

        # Extend the sixteen 32-bit integers into eighty 32-bit integers:
        for j in range(16, 80):
            w[j] = left_rotate(w[j - 3] ^ w[j - 8] ^ w[j - 14] ^ w[j - 16], 1)

        # Initialize hash value for this chunk:
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        # Main loop
        for j in range(80):
            if j <= 19:
                f = d ^ (b & (c ^ d))
                k = 0x5A827999
            elif 20 <= j <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= j <= 59:
                f = (b & c) | (d & (b | c))
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = left_rotate(a, 5) + f + e + k + w[j] & 0xffffffff
            e = d
            d = c
            c = left_rotate(b, 30)
            b = a
            a = temp

        # Add this chunk's hash to result so far:
        h0 = (h0 + a) & 0xffffffff
        h1 = (h1 + b) & 0xffffffff
        h2 = (h2 + c) & 0xffffffff
        h3 = (h3 + d) & 0xffffffff
        h4 = (h4 + e) & 0xffffffff

    # Produce the final hash value (big-endian) as a 160 bit number, hex formatted:
    return "%08x%08x%08x%08x%08x" % (h0, h1, h2, h3, h4)

def sha1_mac(key: bytes, message: bytes) -> bytes:
    return sha1(key + message)

In [22]:
keysize = 16
random_key = os.urandom(keysize)
message = "This is a message to test that our implementation of the SHA1 MAC works properly."

hashed = sha1_mac(random_key, message.encode())

# Verify that I implemented SHA1 correctly
h = hashlib.sha1(random_key + message.encode())

In [23]:
test(hashed == h.hexdigest())

# Challenge 29: Break a SHA-1 keyed MAC using length extension
[Link](https://cryptopals.com/sets/4/challenges/29)

> Secret-prefix SHA-1 MACs are trivially breakable.  
The attack on secret-prefix SHA1 relies on the fact that you can take the ouput of SHA-1 and use it as a new starting point for SHA-1, thus taking an arbitrary SHA-1 hash and "feeding it more data".  
Since the key precedes the data in secret-prefix, any additional data you feed the SHA-1 hash in this fashion will appear to have been hashed with the secret key.  
To carry out the attack, you'll need to account for the fact that SHA-1 is "padded" with the bit-length of the message; your forged message will need to include that padding. We call this "glue padding". The final message you actually forge will be:  
**SHA1(key || original-message || glue-padding || new-message)**  
(where the final padding on the whole constructed message is implied)  
Note that to generate the glue padding, you'll need to know the original bit length of the message; the message itself is known to the attacker, but the secret key isn't, so you'll need to guess at it.  
This sounds more complicated than it is in practice.  
To implement the attack, first write the function that computes the MD padding of an arbitrary message and verify that you're generating the same padding that your SHA-1 implementation is using. This should take you 5-10 minutes.  
Now, take the SHA-1 secret-prefix MAC of the message you want to forge --- this is just a SHA-1 hash --- and break it into 32 bit SHA-1 registers (SHA-1 calls them "a", "b", "c", &c).  
Modify your SHA-1 implementation so that callers can pass in new values for "a", "b", "c" &c (they normally start at magic numbers). With the registers "fixated", hash the additional data you want to forge.  
Using this attack, generate a secret-prefix MAC under a secret key (choose a random word from /usr/share/dict/words or something) of the string:  
**"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"**  
Forge a variant of this message that ends with ";admin=true".

In [24]:
# Imports
import os
import struct

In [25]:
# Given
message = "comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
payload = b";admin=true"

In [26]:
# Generating a pseudo random key, to be run only once.
key = os.urandom(16)

In [27]:
def md_pad(message: bytes) -> bytes:
    """
    Pads the message in accordance with SHA1 padding.
    """
    ml = len(message) * 8
    message += b'\x80'
    while (len(message) * 8) % 512 != 448:
        message += b'\x00'

    message += struct.pack('>Q', ml)
    return message
    
def validate(modified_message: bytes, new_md: bytes) -> bool:
    """
    Verifies the MAC .
    """
    if sha1_mac(key, modified_message) == new_md:
        return True
    return False
    
def sha1_length_extension_attack(message: bytes, original_md: bytes, payload: bytes) -> (bytes, bytes):
    """
    Perform the SHA1 length extension attack.
    """
    for key_length in range(20):
        h = struct.unpack('>5I', bytes.fromhex(original_md))
        modified_message = md_pad(b'A'*key_length + message)[key_length:] + payload
        new_md = sha1(payload, (len(modified_message) + key_length)*8, h[0], h[1], h[2], h[3], h[4])
        if validate(modified_message, new_md):
            print("> Length extension attack successful.")
            return modified_message, new_md
            break

In [28]:
original_md = sha1_mac(key, message.encode())
modified_message, new_md = sha1_length_extension_attack(message.encode(), original_md, payload)

> Length extension attack successful.


In [29]:
test(payload in modified_message)

# Challenge 30: Break an MD4 keyed MAC using length extension
[Link](https://cryptopals.com/sets/4/challenges/30)

> Second verse, same as the first, but use MD4 instead of SHA-1. Having done this attack once against SHA-1, the MD4 variant should take much less time; mostly just the time you'll spend Googling for an implementation of MD4.

In [30]:
# Imports
import os
import struct
import binascii

In [31]:
# Given
message = "comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
payload = b";admin=true"

In [32]:
# Generating a pseudo random key, to be run only once.
key = os.urandom(16)

In [33]:
class MD4:
    """
    This implementation resembles the one of the Wikipedia pseudo-code.
    """
    
    buf = [0x00] * 64

    _F = lambda self, x, y, z: ((x & y) | (~x & z))
    _G = lambda self, x, y, z: ((x & y) | (x & z) | (y & z))
    _H = lambda self, x, y, z: (x ^ y ^ z)

    def __init__(self: object, message: bytes, ml=None, A=0x67452301, B=0xefcdab89, C=0x98badcfe, D=0x10325476):
        self.A, self.B, self.C, self.D = A, B, C, D

        if ml is None:
            ml = len(message) * 8
        length = struct.pack('<Q', ml)

        while len(message) > 64:
            self._handle(message[:64])
            message = message[64:]

        message += b'\x80'
        message += bytes((56 - len(message) % 64) % 64)
        message += length

        while len(message):
            self._handle(message[:64])
            message = message[64:]

    def _handle(self: object, chunk: bytes):
        X = list(struct.unpack('<' + 'I' * 16, chunk))
        A, B, C, D = self.A, self.B, self.C, self.D

        for i in range(16):
            k = i
            if i % 4 == 0:
                A = left_rotate((A + self._F(B, C, D) + X[k]) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._F(A, B, C) + X[k]) & 0xffffffff, 7)
            elif i % 4 == 2:
                C = left_rotate((C + self._F(D, A, B) + X[k]) & 0xffffffff, 11)
            elif i % 4 == 3:
                B = left_rotate((B + self._F(C, D, A) + X[k]) & 0xffffffff, 19)

        for i in range(16):
            k = (i // 4) + (i % 4) * 4
            if i % 4 == 0:
                A = left_rotate((A + self._G(B, C, D) + X[k] + 0x5a827999) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._G(A, B, C) + X[k] + 0x5a827999) & 0xffffffff, 5)
            elif i % 4 == 2:
                C = left_rotate((C + self._G(D, A, B) + X[k] + 0x5a827999) & 0xffffffff, 9)
            elif i % 4 == 3:
                B = left_rotate((B + self._G(C, D, A) + X[k] + 0x5a827999) & 0xffffffff, 13)

        order = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
        for i in range(16):
            k = order[i]
            if i % 4 == 0:
                A = left_rotate((A + self._H(B, C, D) + X[k] + 0x6ed9eba1) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._H(A, B, C) + X[k] + 0x6ed9eba1) & 0xffffffff, 9)
            elif i % 4 == 2:
                C = left_rotate((C + self._H(D, A, B) + X[k] + 0x6ed9eba1) & 0xffffffff, 11)
            elif i % 4 == 3:
                B = left_rotate((B + self._H(C, D, A) + X[k] + 0x6ed9eba1) & 0xffffffff, 15)

        self.A = (self.A + A) & 0xffffffff
        self.B = (self.B + B) & 0xffffffff
        self.C = (self.C + C) & 0xffffffff
        self.D = (self.D + D) & 0xffffffff

    def digest(self: object) -> bytes:
        return struct.pack('<4I', self.A, self.B, self.C, self.D)

    def hex_digest(self: object) -> bytes:
        return binascii.hexlify(self.digest()).decode()

In [34]:
def md_pad(message: bytes) -> bytes:
    """
    Pads the given message the same way the pre-processing of the MD4 algorithm does.
    """
    ml = len(message) * 8

    message += b'\x80'
    message += bytes((56 - len(message) % 64) % 64)
    message += struct.pack('<Q', ml)

    return message
    
def validate(modified_message: bytes, new_md: bytes) -> bool:
    """
    Verifies if the padding is correct.
    """
    if MD4(modified_message).hex_digest() == new_md:
        return True
    return False

In [35]:
def md4_length_extension_attack(message: bytes, original_md: bytes, payload: bytes) -> bytes:
    """
    Performs the length extension attack on an MD4.
    """
    for key_length in range(20):
        h = struct.unpack('<4I', bytes.fromhex(original_md))
        modified_message = md_pad(b'A'*key_length + message)[key_length:] + payload
        new_md = MD4(payload, (len(modified_message) + key_length)*8, h[0], h[1], h[2], h[3]).hex_digest()
        if validate(modified_message, new_md):
            print("> Length extension attack successful.")
            return modified_message, new_md
            break

In [36]:
original_md = MD4(message.encode()).hex_digest()
modified_message, new_md = md4_length_extension_attack(message.encode(), original_md, payload)

> Length extension attack successful.


In [37]:
test(payload in modified_message)

# Challenge 31: Implement and break HMAC-SHA1 with an artificial timing leak
[Link](https://cryptopals.com/sets/4/challenges/31)

> The psuedocode on Wikipedia should be enough. HMAC is very easy.  
Using the web framework of your choosing (Sinatra, web.py, whatever), write a tiny application that has a URL that takes a "file" argument and a "signature" argument, like so:  
**http://localhost:9000/test?file=foo&signature=46b4ec586117154dacd49d664e5d63fdc88efb51**  
Have the server generate an HMAC key, and then verify that the "signature" on incoming requests is valid for "file", using the "==" operator to compare the valid MAC for a file with the "signature" parameter (in other words, verify the HMAC the way any normal programmer would verify it).  
Write a function, call it "insecure_compare", that implements the == operation by doing byte-at-a-time comparisons with early exit (ie, return false at the first non-matching byte).  
In the loop for "insecure_compare", add a 50ms sleep (sleep 50ms after each byte).  
Use your "insecure_compare" function to verify the HMACs on incoming requests, and test that the whole contraption works. Return a 500 if the MAC is invalid, and a 200 if it's OK.  
Using the timing leak in this application, write a program that discovers the valid MAC for any file.

In [38]:
# Imports
import os
import web
import json
import time
import hashlib

In [39]:
# Given
delay = 0.05

In [40]:
class HMAC:
    """
    Computes the HMAC for the hash function given at the time of initialisation.
    This implementation resembles the one of the Wikipedia pseudo-code.
    """
    
    def __init__(self: object, random_key: bytes, hash_func: callable):
        self.hash_func = hash_func
        self.block_size = hash_func().block_size

        if len(random_key) > self.block_size:
            self.key = hash_func(random_key).digest()
        elif len(random_key) < self.block_size:
            self.key = random_key + b'\x00' * (self.block_size-len(random_key))

    def compute(self: object, message: bytes) -> bytes:
        o_key_pad = xor_bytes(self.key, b'\x5c' * self.block_size)
        i_key_pad = xor_bytes(self.key, b'\x36' * self.block_size)
        
        inner_hash = self.hash_func(i_key_pad + message).digest()
        
        return self.hash_func(o_key_pad + inner_hash).hexdigest()

Web server using web.py

In [41]:
urls = (
    '/hello', 'Hello',
    '/test', 'Hash'
)

app = web.application(urls, globals())

HMAC_obj = HMAC(b"YELLOW_SUBMARINE", hashlib.sha1)

class Hello:        
    
    def GET(self):
        params = web.input()
        name = params.name
        if not name:
            name = 'World'
            
        string = "Hello, " + name + "!"
        return {"name" : string}

class Hash:
    
    def _insecure_compare(self, hash1, hash2, delay):
        for b1, b2 in zip(hash1, hash2):
            if b1 != b2:
                return False
            time.sleep(delay)
        return True
    
    def GET(self):
        global HMAC_obj
        params = web.input()
        file = params.file
        signature = params.signature
        delay = params.delay
        
        hmac = HMAC_obj.compute(file.encode())
        if self._insecure_compare(hmac.encode(), signature.encode(), float(delay)):
            return web.HTTPError(200)
        else:
            return web.HTTPError(500)

Test the web server.

In [42]:
response1 = app.request("/hello?name=")
print(response1.data)

response2 = app.request("/hello?name=hexterisk")
print(json.loads(response2.data.decode("utf-8").replace("'",'"')))

b"{'name': 'Hello, World!'}"
{'name': 'Hello, hexterisk!'}


In [43]:
filename = "foo"
signature = "274b7c4d98605fcf739a0bf9237551623f415fb8"
response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature)
print(response)

signature = "8c80a95a8e72b3e822a13924553351a433e267d8"
response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature)
print(response)

<Storage {'status': 500, 'headers': {}, 'header_items': [], 'data': b'500'}>
<Storage {'status': 200, 'headers': {}, 'header_items': [], 'data': b'200'}>


Produces a 160-bit (20-byte) hash value known as a message digest, typically rendered as a hexadecimal number, 40 digits long.    

In [44]:
signature = ""
# We go for twice the size because hexadecimal byte is 2 digits long.
for _ in range(hashlib.sha1().digest_size * 2):
    
    times = []
    # This loop goes over all 16 hexadecimal bytes.
    for i in range(16):
        start = time.time()
        response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature + hex(i)[-1])
        finish = time.time()
        times.append(finish - start)
    signature += hex(times.index(max(times)))[-1]
    print("> Discovered signature:", signature)

response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature + hex(i)[-1])
if response.status == 200:
    print("> Brute force successful.\n> Signature:", signature)
    test(True)
else:
    print("Brute force failed.")
    test(False)    

> Discovered signature: 8
> Discovered signature: 8c
> Discovered signature: 8c8
> Discovered signature: 8c80
> Discovered signature: 8c80a
> Discovered signature: 8c80a9
> Discovered signature: 8c80a95
> Discovered signature: 8c80a95a
> Discovered signature: 8c80a95a8
> Discovered signature: 8c80a95a8e
> Discovered signature: 8c80a95a8e7
> Discovered signature: 8c80a95a8e72
> Discovered signature: 8c80a95a8e72b
> Discovered signature: 8c80a95a8e72b3
> Discovered signature: 8c80a95a8e72b3e
> Discovered signature: 8c80a95a8e72b3e8
> Discovered signature: 8c80a95a8e72b3e82
> Discovered signature: 8c80a95a8e72b3e822
> Discovered signature: 8c80a95a8e72b3e822a
> Discovered signature: 8c80a95a8e72b3e822a1
> Discovered signature: 8c80a95a8e72b3e822a13
> Discovered signature: 8c80a95a8e72b3e822a139
> Discovered signature: 8c80a95a8e72b3e822a1392
> Discovered signature: 8c80a95a8e72b3e822a13924
> Discovered signature: 8c80a95a8e72b3e822a139245
> Discovered signature: 8c80a95a8e72b3e822a1392455

# Challenge 32: Break HMAC-SHA1 with a slightly less artificial timing leak
[Link](https://cryptopals.com/sets/4/challenges/32)

>Reduce the sleep in your "insecure_compare" until your previous solution breaks. (Try 5ms to start.)  
Now break it again.

In [45]:
# Given
delay = 0.005

In [46]:
HMAC_obj = HMAC(b"YELLOW_SUBMARINE", hashlib.sha1)

file = "foo"

Produces a 160-bit (20-byte) hash value known as a message digest, typically rendered as a hexadecimal number, 40 digits long.

In [47]:
signature = ""
for _ in range(hashlib.sha1().digest_size * 2):
# We go for twice the size because hexadecimal byte is 2 digits long.
    times = []
    # This loop goes over all 16 hexadecimal bytes.
    for i in range(16):
        runtime = 0
        # Introduced more rounds so the time difference is prominent
        for _ in range(20):
            start = time.time()
            response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature + hex(i)[-1])
            finish = time.time()
            runtime += finish - start
        times.append(runtime)
    signature += hex(times.index(max(times)))[-1]
    print("> Discovered signature:", signature)

response = app.request("/test?delay=" + str(delay) + "&file=" + filename + "&signature=" + signature + hex(i)[-1])
if response.status == 200:
    print("> Brute force successful.\n> Signature:", signature)
    test(True)
else:
    print("Brute force failed.")
    test(False)

> Discovered signature: 8
> Discovered signature: 8c
> Discovered signature: 8c8
> Discovered signature: 8c80
> Discovered signature: 8c80a
> Discovered signature: 8c80af
> Discovered signature: 8c80aff
> Discovered signature: 8c80aff0
> Discovered signature: 8c80aff0f
> Discovered signature: 8c80aff0f0
> Discovered signature: 8c80aff0f0f


KeyboardInterrupt: 